In [17]:
import numpy as np
import pandas as pd
import math
import plotly.express as px
import statistics
import itertools as it
from functions import mms_enspectra_simulation_functions
import plotly.graph_objects as go

#import pytplot

In [18]:
ensize = len(mms_enspectra_simulation_functions.MMS_ENERGY_BIN)
vsize = 682
vel_set = np.arange(vsize)+10
dist_set = [5, 8.69, 10, 16.85, 20, 35, 50, 60]
dsize = len(dist_set)

In [19]:
result = pd.DataFrame(np.array(np.meshgrid(vel_set, dist_set)).T.reshape(-1,2),columns=["velocity","distance_Re"])

result['distance'] = result['distance_Re'].apply(mms_enspectra_simulation_functions.convert_Re_to_km)
result['energy'] = result['velocity'].apply(mms_enspectra_simulation_functions.calcualte_energy_from_velocity)
result['time'] = result['distance'] / result['velocity'] / 60.
result['energy_binned'] = mms_enspectra_simulation_functions.find_energy_bin_df(result['energy'])

In [20]:
fig = px.line(result, x="time", y="energy_binned", color='distance_Re', log_y=True)
fig.update_xaxes(dtick=5)
fig.show()

In [21]:
target_distances = [8.69,16.85]
for target_distance in target_distances:
    index = result['distance_Re'] == target_distance
    
    fig = px.scatter(result.loc[index,:], x="time", y="energy_binned", log_y=True
                     , title= str(target_distance)+' Re')
    fig.update_xaxes(dtick=5)
    fig.show()

In [22]:
target_distances = [8.69,16.85]
avg_time = 5. 

for target_distance in target_distances:
    index = (result['distance_Re'] == target_distance)
    ntime = round(max(result.loc[index,'time'])/avg_time)
    time_loop = np.arange(0,ntime,1)
    energy_spectra = pd.DataFrame(data = np.empty((ntime,3)), columns =['time','energy','flux'])

    for itime in time_loop:
        st = itime * avg_time
        et = (itime+1) * avg_time
        index = (result['time'] >= st) & (result['time'] < et) & (result['distance_Re'] == target_distance)
        energy_spectra.loc[itime,'flux'] = len(result.loc[index,'energy_binned'])
        if energy_spectra.loc[itime,'flux'] > 0:
            energy_spectra.loc[itime,'energy'] = statistics.mean(result.loc[index,'energy_binned'])
        energy_spectra.loc[itime,'time'] = (st + et)/2
    
    energy_spectra['energy_binned'] = mms_enspectra_simulation_functions.find_energy_bin_df(
        energy_spectra['energy'])

    fig = px.scatter(energy_spectra, x="time", y="energy_binned", color = 'flux', log_y=True
                     , title = 'distance: ' + str(target_distance) + ' Re, ' 
                     + '   average time: ' + str(avg_time) + ' min')
    fig.update_xaxes(dtick=avg_time)
    fig.show()


In [ ]:
target_distances = [8.69, 16.85]
avg_time = 5. 

for target_distance in target_distances:
    index = (result['distance_Re'] == target_distance)
    ntime = round(max(result.loc[index,'time'])/avg_time)
    time_loop = np.arange(0,ntime,1)
    energy_spectra = pd.DataFrame(np.array(
        np.meshgrid(time_loop, mms_enspectra_simulation_functions.MMS_ENERGY_BIN)).T.reshape(-1,2)
                                  , columns=["itime","energy_binned"])
    
    energy_spectra['time'] = (energy_spectra['itime']+0.5)*avg_time
    energy_spectra['flux'] = energy_spectra.apply(
        lambda x: mms_enspectra_simulation_functions.calculate_flux(x['itime'], x['energy_binned'], result, target_distance, avg_time), axis=1)      
    index_valid = (energy_spectra['flux'] > 0)

    beam = pd.DataFrame(np.empty((ntime,2)), columns=['time','energy_binned'])
    i = 0
    for itime in np.unique(energy_spectra['time']):
        index = energy_spectra['time'] == itime
        beam.loc[i,'time'] = itime
        if energy_spectra.loc[index,'flux'].max() == 0:
            beam.loc[i, 'energy_binned'] = None
        else:
            beam.loc[i, 'energy_binned'] = energy_spectra.loc[(energy_spectra.loc[index,'flux']).idxmax(),'energy_binned']
        i = i + 1
              
    dispersion = pd.DataFrame(np.empty((ntime,2)), columns=['time','energy_binned'])
    dispersion = beam
    dispersion['energy_binned'] = None
    dispersion['decrease'] = (-dispersion['energy_binned'].diff()) >= 0

    for iindex in dispersion['time'].index:
        start_index = 0
        end_index = 0
        if start_index == 0:
            length = 1
            continue
        if dispersion['decrease'] == True:
            length + 1
            continue
        else:
            if length > 3:
                end_index = iindex - 1
                dispersion.loc[start_index:end_index, 'energy_binned'] = beam['energy_binned']
                
            start_index = 0
    
    
    
    fig = px.scatter(energy_spectra.loc[index_valid,:], x="time", y="energy_binned", color = 'flux', log_y = True
                     , title = 'distance: ' + str(target_distance) + ' Re, ' 
                     + '   average time: ' + str(avg_time) + ' min')
    
    reference_line = go.Scatter(x = list(beam['time']), y = list(beam['energy_binned'])
                                , mode = "lines", line = go.scatter.Line(color="gray"), showlegend=False)
        
    reference_line = go.Scatter(x = list(dispersion['time']), y = list(dispersion['energy_binned'])
                                , mode = "lines", line = go.scatter.Line(color="red"), showlegend=False)    
        
    fig.add_trace(reference_line)

    fig.update_xaxes(dtick=avg_time)
    fig.show()


In [74]:
dispersion['decrease'] = (-dispersion['energy_binned'].diff()) >= 0
dispersion['decrease']

0     False
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10     True
11     True
12     True
13     True
14     True
15     True
16     True
17     True
18     True
19     True
20    False
21    False
22     True
23     True
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
Name: decrease, dtype: bool

In [ ]:
new_energy_spectra = energy_spectra.pivot(index='energy_binned', columns='time', values='flux')

fig = px.imshow(np.array(new_energy_spectra)
                , x = list(np.unique(new_energy_spectra['time']))
                , y = list(np.unique(new_energy_spectra['energy_binned']))
                , title = 'distance: '+str(target_distance)+' Re, '+'   average time: '+str(avg_time)+' min')
#fig.update_xaxes(dtick=avg_time)
fig.show()

In [ ]:
(new_energy_spectra)

In [ ]:
#pytplot.store_data("enspectra"
#                   , data={  'x': np.unique(energy_spectra['time'])
#                           , 'y': np.unique(energy_spectra['flux']) 
#                           , 'v': energy_spectra['energy_binned']})